<a href="https://colab.research.google.com/github/genie0320/langchain/blob/main/04_RAG_advance_multiQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요

RAG는 여러 과정을 거치면서 진행된다. 매 과정이 치명적으로 중요하다.

- 멀티쿼리 : 대충질문해도 좋은 답변을 원하는 이들을 위해 다음에 집중해야 한다.
- 페어런트 도큐먼트 : 앞뒤 문장을 잘 담아야 하고(chunk의 중요성)
- 셀프쿼리(질문재해석) : 시맨틱검색 말고 쿼리가 필요한 경우
  - 시맨틱검색이란 질문문장과 임베딩데이터의 벡터값에 따라 유사데이터를 걸러내는 것인데, 이 경우 질문의 모양이 조금만 달라져도 추출 데이터 자체가 달라진다. 이것은 '질문이 우선되는 구조'이기 때문이다.
  이 경우, 데이터를 중심으로 사용자의 질문을 참고하여 쿼리를 날려서 정리해야 할 필요가 생길 수 있다. 이것도 고려해야 한다.
- 타임 웨이티드 : 오래된 자료는 덜 참고했으면...
  - 최근에 올라간 자료에 무게를 둬서 답변을 참고했으면 좋겠다.

## 멀티쿼리 리트리버

사용자의 질문을 여러개의 유사질문으로 재생성하고, 각 질문에 대한 데이터를 추출해서 답변을 생성하는 방법.

# 코드

## Setting

In [1]:
# # colab 환경설정
# import os
# from google.colab import userdata

# os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
# os.environ['HF_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
# os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# !pip install -U -q langchain pypdf sentence_transformers chromadb langchain-openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.


## Multi-Query Retriever

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader

from langchain.embeddings import HuggingFaceEmbeddings

In [6]:
# url = 'https://brunch.co.kr/@hae-ra/22'
url = 'https://n.news.naver.com/article/002/0002319323?cds=news_media_pc&type=editn'
loader = WebBaseLoader(url)
documents = loader.load()

In [7]:
documents[0].page_content[:50]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n헤어샵 포기 후 택한 공장, 일이 불규칙해지더니 서명지가'

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    is_separator_regex=False,
    )
texts = text_splitter.split_documents(documents)

In [9]:
texts[0]

Document(page_content='헤어샵 포기 후 택한 공장, 일이 불규칙해지더니 서명지가 돌았다[전수경의 MZ 여성 그리고 빈곤]\n\n\n\n\n\n\n\n\n\n\n\n\n\n본문 바로가기\n\n\n\n\n\n\n이전 페이지\n\n\n\n\n\n\n프레시안\n\n\n\n구독\n\n언론사를 구독하면 메인에서 바로 볼 수 있어요!\n\n\n메인 뉴스판에서 프레시안 주요뉴스를  볼 수 있습니다.\n보러가기\n\n\n프레시안 언론사 구독 해지되었습니다.\n\n\n\n\n\n\n\n\n\n\n주요뉴스\n총선\n정치\n경제\n사회\n생활\n세계\n사설/칼럼\n랭킹\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPICK\n안내\n\n\n언론사가 주요기사로선정한 기사입니다.\n언론사별 바로가기\n닫기\n\n\n\n\n헤어샵 포기 후 택한 공장, 일이 불규칙해지더니 서명지가 돌았다[전수경의 MZ 여성 그리고 빈곤]\n\n\n\n\n입력2024.02.14. 오전 5:04\n\n기사원문\n \n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇', metadata={'source': 'https://n.news.naver.com/article/002/0002319323?cds=news_media_pc&type=editn', 'title': '헤어샵 포기 후 택한 공장, 일이 불규칙해지더니 서명지가 돌았다[전수경의 MZ 여성 그리고 빈곤]', 'language': 'ko'})

In [ ]:
DB_URL = '/content/drive/MyDrive/vector_upgrade/'
MODEL_NAME= "jhgan/ko-sbert-nli"
encode_kwargs={'normalize_embeddings':True}

ko_embed = HuggingFaceEmbeddings(
    model_name= MODEL_NAME,
    # model_kwargs={'device' : 'cpu'},
    encode_kwargs=encode_kwargs
    )
# vector_stores = Chroma.from_documents(texts, hug_embed)
vector_stores = Chroma.from_documents(
    texts,
    ko_embed,
    persist_directory = DB_URL
)
print('hug_embed : Document Stored!!')

load_vector_store = Chroma(
    persist_directory = DB_URL,
    embedding_function = ko_embed,
    )

print(f'Vector store seted : load_vector_store')

In [13]:
# ___ setting ___
LLM_MODEL = "gpt-3.5-turbo"
MAX = 265
TEMP = 1.5
# _______________

from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

openai_compbot = OpenAI(
    temperature=TEMP,
    max_tokens = MAX,
    verbose = True,
)
openai_chatbot = ChatOpenAI(
    model_name = LLM_MODEL,
    temperature=TEMP,
    max_tokens = MAX,
    verbose = True,
)

In [15]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI

question = '헤어샵을 포기한 이유는?'

llm = openai_chatbot
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=load_vector_store.as_retriever(),
    llm=llm
)

In [18]:
import logging

logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [19]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 헤어샵을 왜 그만두게 되었나요?', '2. 헤어샵을 나가야만 했던 이유가 어떻게 되시나요?', '3. 헤어샵을 더 이상 이용하지 않게 된 이유가 무엇인가요?']


4

In [20]:
unique_docs

[Document(page_content='없었다, 헤어용품 판매, 정기이용권 판매 압박도 부담이었다. 헤어샵의 매출에서 디자이너가 판매하는 정기이용권은 비중이 크다. 정기이용권을 선불로 구매한 고객들은 자신이 원하는 디자이너에게 계속 머리를 맡기고 싶어한다. 내 스타일을 잘 아는 디자이너 대신 낯선 디자이너가 내 머리를 손질한다면 고객은 헤어샵에 환불을 요구할 수도 있다. 코로나바이러스가 퍼지기 시작하던 때 헤어샵을 찾는 사람들이 급속하게 줄어들면서 매출이 바닥을 찍었다. 디자이너들이 받아 갈 급여가 거의 안 나왔다. 헤어샵 원장과 헤어샵 체인의 소유주는 고객이 선불로 지불한 이용권을 구실로 디자이너들을 잡았다. 디자이너가 나가려면 고객이 환불을 요구할 금액을 물어내야 했다. 선불권의 조건에 특정 디자이너가 아니어도 된다는 문구가 있지만 샵의 입장에서 고객이 환불을 요구할 때 거부하기는 어려울 것이다. 거액의 선불 이용권이 자신 앞으로 달려 있을 때 디자이너가 선택할 수 있는 건 돈을 물어주고 나오거나, 돈을 물어주기를', metadata={'language': 'ko', 'source': 'https://n.news.naver.com/article/002/0002319323?cds=news_media_pc&type=editn', 'title': '헤어샵 포기 후 택한 공장, 일이 불규칙해지더니 서명지가 돌았다[전수경의 MZ 여성 그리고 빈곤]'}),
 Document(page_content="어머니는 나이가 들면서 몸이 자꾸 아프다. 집에서 벗어나고자 스스로 직업을 개척했고 능력 있는 헤어디자이너도 됐지만 자리를 잡지 못했다. 헤어 일이 여전히 좋아 손이 굳을까 봐 걱정이지만 다시 할 수 있을지는 모르겠다. 공장에서 다친 허리는 끌어안고 있다가 나중에 정형외과에 갔다. 도수치료라는 것을 받으라고 권유해서 해보니 한 번에 12만 원이 나왔다. 지금 월급으로는 불가능했다. '청년여성 산재회복 지원사업'에서 받은 100만 원의 절반을 니은은 부모님의 병원비로 부